In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
! git clone https://github.com/Tikquuss/meta_XLM /content/meta_XLM

Cloning into '/content/meta_XLM'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 506 (delta 185), reused 157 (delta 79), pack-reused 235
Receiving objects: 100% (506/506), 30.45 MiB | 18.56 MiB/s, done.
Resolving deltas: 100% (231/231), done.


In [0]:
! cp -R /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs /content

In [4]:
%bookmark HOME "/content/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /content/meta_XLM/XLM
/content/meta_XLM/XLM


In [5]:
%cd tools
! git clone https://github.com/moses-smt/mosesdecoder
! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/meta_XLM/XLM/tools
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 147544 (delta 12), reused 15 (delta 5), pack-reused 147514
Receiving objects: 100% (147544/147544), 129.75 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (113998/113998), done.
Cloning into 'fastBPE'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 59 (delta 0), reused 2 (delta 0), pack-reused 54
Unpacking objects: 100% (59/59), done.


In [6]:
%cd -b HOME

(bookmark:HOME) -> /content/meta_XLM/XLM
/content/meta_XLM/XLM


In [0]:
! mkdir data

# **Bafia - Bulu**

In [8]:
%env csv_path=/content
%env output_dir=data
%env data_type=para
%env languages=Bafia,Bulu
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

env: csv_path=/content
env: output_dir=data
env: data_type=para
env: languages=Bafia,Bulu
Bafia - Bulu
======= Read 7950 totals samples
======= Delete 4 samples
======= Save 7946 samples
Bulu
======= Read 31297 totals samples
======= Delete 2402 samples
======= Save 28895 samples


In [0]:
# comme il y a une partie de données mono disponible pour l'une des langues, on genre une fois pour l'autre
! cp data/para/Bafia-Bulu/Bafia-Bulu.Bafia.txt  data/mono
! mv data/mono/Bafia-Bulu.Bafia.txt data/mono/Bafia.txt
# put everything in one place
! cp data/mono/Bulu/Bulu.txt data/mono
! rm -R data/mono/Bulu

In [10]:
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=data/para/Bafia-Bulu      
%env MONO_PATH=data/mono   
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=data/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Bafia-Bulu:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=data/para/Bafia-Bulu
env: MONO_PATH=data/mono
env: SAME_VOCAB=True
env: nCodes=10000
env: shuf_n_samples=1000000
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=data/processed
env: n_samples=-1
env: sub_task=Bafia-Bulu:-1


In [11]:
%env add_para_data_to_mono_data=False
! ../build_meta_data.sh $sub_task $n_samples $add_para_data_to_mono_data

env: add_para_data_to_mono_data=False
params ok !
*** Cleaning and tokenizing Bafia-Bulu data ... ***
Tokenizer Version 1.1
Language: Bafia
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bafia data to data/para/Bafia-Bulu/? ***
Tokenizer Version 1.1
Language: Bulu
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bulu data to data/para/Bafia-Bulu/? ***
Tokenizer Version 1.1
Language: Bafia
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia data to data/mono/? ***
Tokenizer Version 1.1
Language: Bulu
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bulu data to data/mono/? ***


*** split into train / valid / test ***



***build the training set for BPE tokenization (10000 codes)***


***shuf ... Generating 1000000 random permutations of training data and store result in data/processed/Bafia-Bulu/bpe.train***


***Learn the BPE vocabulary on the training set : data/processed/bpe.train ...***


In [12]:
%env OUTPATH=data/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=data/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [0]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
  i = tensor_len
  while True :
    if i%batch_size == 0 :
      return i//batch_size
    i = i + 1

import io

def n_lines(file_path):
  return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [14]:
tensor_len = n_lines(file_path = "data/processed/Bafia-Bulu.Bafia.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6360


In [15]:
%env epoch_size=6360

%env lgs=Bafia-Bulu

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6360
env: lgs=Bafia-Bulu
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [16]:
%env mlm_steps=Bafia,Bulu,Bafia-Bulu
%env batch_size=8
%env max_epoch=100
! python train.py --exp_name mlm_tlm_BafiaBulu --exp_id $exp_id --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
                                           (lin2): Linear(in_features=4096, out_features=1024, bias=True)
                                         )
                                         (1): TransformerFFN(
                                           (lin1): Linear(in_features=1024, out_features=4096, bias=True)
                                           (lin2): Linear(in_features=4096, out_features=1024, bias=True)
                                         )
                                         (2): TransformerFFN(
                                           (lin1): Linear(in_features=1024, out_features=4096, bias=True)
                                           (lin2): Linear(in_features=4096, out_features=1024, bias=True)
                                         )
                                         (3): TransformerFFN(
                                           (lin1): Linear(in_features=1024, o

In [17]:
! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

sending incremental file list
mlm_tlm_BafiaBulu/
mlm_tlm_BafiaBulu/maml/
mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
    343,595,658 100%   32.62MB/s    0:00:10 (xfr#1, to-chk=8/11)
mlm_tlm_BafiaBulu/maml/checkpoint.pth
  1,029,961,422 100%  126.95MB/s    0:00:07 (xfr#2, to-chk=7/11)
mlm_tlm_BafiaBulu/maml/params.pkl
          3,001 100%    4.03kB/s    0:00:00 (xfr#3, to-chk=6/11)
mlm_tlm_BafiaBulu/maml/train.log
        756,225 100%  611.34kB/s    0:00:01 (xfr#4, to-chk=5/11)
mlm_tlm_BafiaBulu/maml/hypotheses/
mlm_tlm_BafiaBulu/maml/hypotheses/ref.Bafia-Bulu.test.txt
         90,641 100%    6.17MB/s    0:00:00 (xfr#5, to-chk=3/11)
mlm_tlm_BafiaBulu/maml/hypotheses/ref.Bafia-Bulu.valid.txt
         90,425 100%    3.19MB/s    0:00:00 (xfr#6, to-chk=2/11)
mlm_tlm_BafiaBulu/maml/hypotheses/ref.Bulu-Bafia.test.txt
        136,797 100%    2.96MB/s    0:00:00 (xfr#7, to-chk=1/11)
mlm_tlm_BafiaBulu/maml/hypotheses/ref.Bulu-Bafia.valid.txt
        135,383 100%    2.27MB/s    0:00:00 (xfr#8, 

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [18]:
%env epoch_size=6360
%env lgs=Bafia-Bulu

env: epoch_size=6360
env: lgs=Bafia-Bulu


In [19]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [24]:
%env stopping_criterion=valid_mt_bleu,10
%env validation_metrics=valid_mt_bleu
%env reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
%env ae_steps=Bafia,Bulu
%env bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu

env: stopping_criterion=valid_mt_bleu,10
env: validation_metrics=valid_mt_bleu
env: reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,Bulu
env: bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu


In [26]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
! python train.py --exp_name UnSupMT_BafiaBulu --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 8911f7acdb62
INFO - 05/18/20 16:18:27 - 0:00:00 - ============ Initialized logger ============
INFO - 05/18/20 16:18:27 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Bafia', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
                                     bptt: 256
                   

In [0]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [29]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=1
# supervised MT
%env mt_steps=Bafia-Bulu,Bulu-Bafia          
! python train.py --exp_name MT_BafiaBulu --exp_id $exp_id  --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
env: batch_size=1
env: mt_steps=Bafia-Bulu,Bulu-Bafia
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 8911f7acdb62
INFO - 05/18/20 16:27:49 - 0:00:00 - ============ Initialized logger ============
INFO - 05/18/20 16:27:49 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Bafia', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 1
                                     beam_size: 1
            

In [0]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [0]:
d=[]
while(1):
  d.append(d)